In [1]:
import os
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_openai import ChatOpenAI , OpenAIEmbeddings
from langchain_core.documents import Document
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector
from langchain.chains import GraphCypherQAChain
from langchain.chains import RetrievalQA
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)
os.environ['LANGCHAIN_TRACKING_V2']='true'
os.environ['LANGCHAIN_API_KEY']=os.getenv('LANGCHAIN_API_KEY')

In [2]:
os.environ["NEO4J_URI"] = os.getenv("NEO4J_URI")
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = os.getenv("NEO4J_PASSWORD")
api_version = "2023-07-01-preview"

In [3]:
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-0125")
llm_transformer = LLMGraphTransformer(llm=llm)

In [4]:
text = """
Mr. and Mrs. Dursley, of number four, Privet Drive, were proud to say
that they were perfectly normal, thank you very much. They were the last
people you'd expect to be involved in anything strange or mysterious,
because they just didn't hold with such nonsense.
Mr. Dursley was the director of a firm called Grunnings, which made
drills. He was a big, beefy man with hardly any neck, although he did
have a very large mustache. Mrs. Dursley was thin and blonde and had
nearly twice the usual amount of neck, which came in very useful as she
spent so much of her time craning over garden fences, spying on the
neighbors. The Dursleys had a small son called Dudley and in their
opinion there was no finer boy anywhere.
The Dursleys had everything they wanted, but they also had a secret, and
their greatest fear was that somebody would discover it. They didn't
think they could bear it if anyone found out about the Potters. Mrs.
Potter was Mrs. Dursley's sister, but they hadn't met for several years;
in fact, Mrs. Dursley pretended she didn't have a sister, because her
sister and her good-for-nothing husband were as unDursleyish as it was
possible to be. The Dursleys shuddered to think what the neighbors would
say if the Potters arrived in the street. The Dursleys knew that the
Potters had a small son, too, but they had never even seen him. This boy
was another good reason for keeping the Potters away; they didn't want
Dudley mixing with a child like that.
"""
documents = [Document(page_content=text)]
graph_documents = llm_transformer.convert_to_graph_documents(documents)
print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

Nodes:[Node(id='Mr. And Mrs. Dursley', type='Person'), Node(id='Mrs. Dursley', type='Person'), Node(id='Mr. Dursley', type='Person'), Node(id='Dudley', type='Person'), Node(id='Mrs. Potter', type='Person')]
Relationships:[Relationship(source=Node(id='Mr. And Mrs. Dursley', type='Person'), target=Node(id='Dudley', type='Person'), type='PARENT'), Relationship(source=Node(id='Mrs. Dursley', type='Person'), target=Node(id='Dudley', type='Person'), type='PARENT'), Relationship(source=Node(id='Mr. Dursley', type='Person'), target=Node(id='Dudley', type='Person'), type='PARENT'), Relationship(source=Node(id='Mrs. Dursley', type='Person'), target=Node(id='Mrs. Potter', type='Person'), type='SISTER')]


In [5]:
graph = Neo4jGraph()
graph.add_graph_documents(
  graph_documents, 
  baseEntityLabel=True, 
  include_source=True
)

In [6]:
embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002"
)
vector_index = Neo4jVector.from_existing_graph(
    embeddings,
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

In [7]:
query = "Who is Dudley?"

results = vector_index.similarity_search(query, k=1)
print(results[0].page_content)


text: 
Mr. and Mrs. Dursley, of number four, Privet Drive, were proud to say
that they were perfectly normal, thank you very much. They were the last
people you'd expect to be involved in anything strange or mysterious,
because they just didn't hold with such nonsense.
Mr. Dursley was the director of a firm called Grunnings, which made
drills. He was a big, beefy man with hardly any neck, although he did
have a very large mustache. Mrs. Dursley was thin and blonde and had
nearly twice the usual amount of neck, which came in very useful as she
spent so much of her time craning over garden fences, spying on the
neighbors. The Dursleys had a small son called Dudley and in their
opinion there was no finer boy anywhere.
The Dursleys had everything they wanted, but they also had a secret, and
their greatest fear was that somebody would discover it. They didn't
think they could bear it if anyone found out about the Potters. Mrs.
Potter was Mrs. Dursley's sister, but they hadn't met for sever

### CypherChain

In [8]:
chain = GraphCypherQAChain.from_llm(graph=graph, llm=llm, verbose=True)
response = chain.invoke({"query": "What is Mr. Dursley's job?"})
response




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {id: "Mr. Dursley"})-[:PARENT]->(child)
RETURN child.job
Full Context:
[{'child.job': None}]

> Finished chain.


{'query': "What is Mr. Dursley's job?", 'result': "I don't know the answer."}

### QAChain

In [10]:
qa_chain = RetrievalQA.from_chain_type(
    llm, retriever=vector_index.as_retriever()
)

result = qa_chain.invoke({"query": "What is Mr. Dursley's job?"})
result["result"]

"Mr. Dursley's job is the director of a firm called Grunnings, which makes drills."

## French Revolution GraphRAG

In [11]:
from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import Tuple, List, Optional
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain.document_loaders import WikipediaLoader
from langchain.text_splitter import TokenTextSplitter
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from langchain_core.runnables import ConfigurableField, RunnableParallel, RunnablePassthrough

In [13]:
raw_documents = WikipediaLoader(query="French Revolution").load()
raw_documents

c:\Users\KIIT\anaconda3\envs\myenv\Lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file c:\Users\KIIT\anaconda3\envs\myenv\Lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


[Document(page_content="The French Revolution was a period of political and societal change in France that began with the Estates General of 1789, and ended with the coup of 18 Brumaire in November 1799 and the formation of the French Consulate. Many of its ideas are considered fundamental principles of liberal democracy, while its values and institutions remain central to modern French political discourse.\nIts causes are generally agreed to be a combination of social, political and economic factors, which the Ancien Régime proved unable to manage. A financial crisis and widespread social distress led in May 1789 to the convocation of the Estates General, which was converted into a National Assembly in June. The Storming of the Bastille on 14 July led to a series of radical measures by the Assembly, among them the abolition of feudalism, state control over the Catholic Church in France, and a declaration of rights.\nThe next three years were dominated by the struggle for political con

In [14]:
text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)
documents = text_splitter.split_documents(raw_documents)

In [21]:
documents[:24]

[Document(page_content='The French Revolution was a period of political and societal change in France that began with the Estates General of 1789, and ended with the coup of 18 Brumaire in November 1799 and the formation of the French Consulate. Many of its ideas are considered fundamental principles of liberal democracy, while its values and institutions remain central to modern French political discourse.\nIts causes are generally agreed to be a combination of social, political and economic factors, which the Ancien Régime proved unable to manage. A financial crisis and widespread social distress led in May 1789 to the convocation of the Estates General, which was converted into a National Assembly in June. The Storming of the Bastille on 14 July led to a series of radical measures by the Assembly, among them the abolition of feudalism, state control over the Catholic Church in France, and a declaration of rights.\nThe next three years were dominated by the struggle for political con

In [15]:

import pickle

# Save the documents variable
with open('documents.pkl', 'wb') as f:
    pickle.dump(documents, f)

In [22]:
llm_transformer = LLMGraphTransformer(llm=llm)
graph_documents = llm_transformer.convert_to_graph_documents(documents[:24])
print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

Nodes:[Node(id='French Revolution', type='Event'), Node(id='Estates General Of 1789', type='Event'), Node(id='Coup Of 18 Brumaire', type='Event'), Node(id='French Consulate', type='Event'), Node(id='Liberal Democracy', type='Concept'), Node(id='Modern French Political Discourse', type='Concept'), Node(id='Social Factors', type='Concept'), Node(id='Political Factors', type='Concept'), Node(id='Economic Factors', type='Concept'), Node(id='Ancien Régime', type='Concept'), Node(id='Financial Crisis', type='Concept'), Node(id='Social Distress', type='Concept'), Node(id='Convocation Of The Estates General', type='Event'), Node(id='National Assembly', type='Event'), Node(id='Storming Of The Bastille', type='Event'), Node(id='Abolition Of Feudalism', type='Event'), Node(id='State Control Over The Catholic Church In France', type='Event'), Node(id='Declaration Of Rights', type='Event'), Node(id='French Revolutionary Wars', type='Event'), Node(id='Insurrection Of 10 August 1792', type='Event'), 

In [23]:

graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

In [24]:
vector_index = Neo4jVector.from_existing_graph(
    embeddings,
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

### QAChain

In [31]:
qa_graph_chain = RetrievalQA.from_chain_type(
    llm, retriever=vector_index.as_retriever(), verbose = True, return_source_documents=True
)

result = qa_graph_chain({"query": "Considering the financial crisis and resistance to reform by the Ancien Régime, how did the convocation of the Estates General in May 1789 specifically address these multifaceted issues?"})
result["result"]



> Entering new RetrievalQA chain...

> Finished chain.


'The convocation of the Estates General in May 1789 aimed to address the financial crisis and resistance to reform by the Ancien Régime by bringing together representatives from the three estates of the realm: the clergy, the nobility, and the commoners (Third Estate). The hope was that by having all three estates present, they could work together to find solutions to the financial crisis and implement necessary reforms. However, the specific structure and voting methods of the Estates General led to disagreements and ultimately the formation of the National Assembly by the Third Estate, which played a significant role in the subsequent events of the French Revolution.'

In [32]:
result

{'query': 'Considering the financial crisis and resistance to reform by the Ancien Régime, how did the convocation of the Estates General in May 1789 specifically address these multifaceted issues?',
 'result': 'The convocation of the Estates General in May 1789 aimed to address the financial crisis and resistance to reform by the Ancien Régime by bringing together representatives from the three estates of the realm: the clergy, the nobility, and the commoners (Third Estate). The hope was that by having all three estates present, they could work together to find solutions to the financial crisis and implement necessary reforms. However, the specific structure and voting methods of the Estates General led to disagreements and ultimately the formation of the National Assembly by the Third Estate, which played a significant role in the subsequent events of the French Revolution.',
 'source_documents': [Document(page_content='\ntext: During the French Revolution, the National Assembly (Fre

In [26]:
# Defining each question as a variable
question1 = "How did the economic policies of the Ancien Régime contribute to the financial crisis that precipitated the French Revolution?"
question2 = "In what ways did the social and political structure of the Estates-General contribute to its transformation into the National Assembly?"
question3 = "What role did economic depression and military defeats play in the radicalization of the French Revolution in 1792?"
question4 = "How did the French Revolutionary Wars affect the internal political landscape of France from 1792 to 1799?"
question5 = "Examine the socio-economic reasons behind the calling of the Estates-General in 1789."
question6 = "How did Enlightenment ideas influence the legislative reforms of the National Assembly?"
question7 = "What event directly led to the transformation of the Estates-General into the National Assembly in June 1789?"
question8 = "Which radical measure taken by the National Assembly on July 14, 1789, symbolically marked the beginning of the French Revolution?"
question9 = "Which governing body replaced the National Convention after the fall of Robespierre in 1794?"
question10 = "What significant political change occurred in France on 18 Brumaire in 1799?"
question11 = "Considering the financial difficulties faced by the Ancien Régime, how did the complex and inconsistent tax system contribute to the financial instability and eventual calling of the Estates-General?"
question12 = "What role did the socio-economic pressures such as the increase in the population and the widening gap between the rich and the poor play in setting the stage for the French Revolution?"
question13 = "How did the financial crisis, exacerbated by poor harvests and high food prices, lead to the convening of the Estates-General in 1789?"
question14 = "Discuss the immediate political repercussions of the Storming of the Bastille on the French Revolution."

# Creating a list of all questions
questions = [question1, question2, question3, question4, question5, question6, question7, question8, question9, question10, question11, question12, question13, question14]



In [33]:
graph_results = []
graph_source_documents = []
for q in questions:
    graph_results.append(qa_graph_chain({"query": q})["result"])
    graph_source_documents.append(qa_graph_chain({"query": q})["source_documents"])



> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA ch

In [34]:
import pandas as pd

# Assuming questions, rag_results, graph_results are your lists
df = pd.DataFrame({
    'questions': questions,
    'graph_results': graph_results,
    'graph_source_documents': graph_source_documents
})

df.head(20)

,questions,graph_results,graph_source_documents
0,How did the economic policies of the Ancien Ré...,The economic policies of the Ancien Régime con...,"[page_content=""\ntext: % of the population by ..."
1,In what ways did the social and political stru...,The social and political structure of the Esta...,[page_content='\ntext: During the French Revol...
2,What role did economic depression and military...,Economic depression and military defeats playe...,"[page_content=""\ntext: % of the population by ..."
3,How did the French Revolutionary Wars affect t...,The French Revolutionary Wars had a significan...,[page_content='\ntext: The French Revolutionar...
4,Examine the socio-economic reasons behind the ...,The calling of the Estates-General in 1789 was...,"[page_content=""\ntext: % of the population by ..."
5,How did Enlightenment ideas influence the legi...,Enlightenment ideas heavily influenced the leg...,[page_content='\ntext: During the French Revol...
6,What event directly led to the transformation ...,The event that directly led to the transformat...,[page_content='\ntext: The following is a time...
7,Which radical measure taken by the National As...,The radical measure taken by the National Asse...,[page_content='\ntext: The French Revolution w...
8,Which governing body replaced the National Con...,The governing body that replaced the National ...,[page_content='\ntext: During the French Revol...
9,What significant political change occurred in ...,"On 18 Brumaire in 1799, the significant politi...",[page_content='\ntext: The French Revolution o...


In [35]:
graphrag_text = []
for index, row in df.iterrows():
    graphrag_documents = row['graph_source_documents']
    combined_graphrag_documents = " ".join(doc.page_content for doc in graphrag_documents)
    graphrag_text.append(combined_graphrag_documents)


df.graph_source_documents = graphrag_text
df.head()

,questions,graph_results,graph_source_documents
0,How did the economic policies of the Ancien Ré...,The economic policies of the Ancien Régime con...,\ntext: % of the population by 1789. Although ...
1,In what ways did the social and political stru...,The social and political structure of the Esta...,"\ntext: During the French Revolution, the Nati..."
2,What role did economic depression and military...,Economic depression and military defeats playe...,\ntext: % of the population by 1789. Although ...
3,How did the French Revolutionary Wars affect t...,The French Revolutionary Wars had a significan...,\ntext: The French Revolutionary Wars (French:...
4,Examine the socio-economic reasons behind the ...,The calling of the Estates-General in 1789 was...,\ntext: % of the population by 1789. Although ...


In [36]:

print('QUESTION')
print(df.iloc[0]['questions'])
#printing whitespace
print('')
print('ANSWER')
print(df.iloc[0]['graph_results'])
print('')
print('CONTEXT')
print(df.iloc[0]['graph_source_documents'])

QUESTION
How did the economic policies of the Ancien Régime contribute to the financial crisis that precipitated the French Revolution?

ANSWER
The economic policies of the Ancien Régime contributed to the financial crisis that precipitated the French Revolution in several ways. The state's debt crisis was exacerbated by varying tax rates across regions, inconsistent tax collection, and the complexity of the tax system. This led to uncertainty over the actual tax contributions and caused resentment among taxpayers. Attempts to reform and simplify the system were blocked by regional Parlements, which approved financial policies. The resulting impasse and the inability to effectively manage the economic crisis, along with other social, political, and cultural factors, ultimately led to the revolution.

CONTEXT

text: % of the population by 1789. Although the 18th century was a period of increasing prosperity, the benefits were distributed unevenly across regions and social groups. Those 

In [37]:
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain

groundedness_critique_prompt = PromptTemplate.from_template("""
You will be given a context and answer about that context.
Your task is to provide a 'total rating' scoring how well the ANSWER is entailed by the CONTEXT. 
Give your answer on a scale of 1 to 5, where 1 means that the ANSWER is logically false from the information contained in the CONTEXT, and 5 means that the ANSWER follows logically from the information contained in the CONTEXT.

Provide your response in a list as follows:

Response:::
[Evaluation: (your rationale for the rating, as a text),
Total rating: (your rating, as a number between 1 and 5)]

You MUST provide values for 'Evaluation:' and 'Total rating:' in your response.

Now here are the context, question and answer.

Context: {context}\n
Answer: {answer}\n
Response::: """)

relevance_critique_prompt = PromptTemplate.from_template("""
You will be given a context, and question and answer about that context.
Your task is to provide a 'total rating' to measure how well the answer addresses the main aspects of the question, based on the context. 
Consider whether all and only the important aspects are contained in the answer when evaluating relevance. 
Given the context and question, score the relevance of the answer between one to five stars using the following rating scale: 

Give your response on a scale of 1 to 5, where 1 means that the answer doesn't address the question at all, and 5 means that the answer is perfectly matching the question.

Provide your response in a list as follows:

Response:::
[Evaluation: (your rationale for the rating, as a text),
Total rating: (your rating, as a number between 1 and 5)]

You MUST provide values for 'Evaluation:' and 'Total rating:' in your response.

Now here is the question.

Answer: {answer}\n
Question: {question}\n
Context: {context}\n
Response::: """)

coherence_critique_prompt = PromptTemplate.from_template("""
You will be given a question and answer.
Your task is to measure the coherence of the answer. Coherence is measured by how well all the sentences fit together and sound naturally as a whole. Consider the overall quality of the answer when evaluating coherence. 
Given the question and answer, score the coherence of answer on a scale of 1 to 5, where 1 means that the answer completely lacks coherence, 5 means that the answer has perfect coherency.

Provide your response in a list as follows:

Response:::
[Evaluation: (your rationale for the rating, as a text),
Total rating: (your rating, as a number between 1 and 5)]

You MUST provide values for 'Evaluation:' and 'Total rating:' in your response.

Now here is the question.

Question: {question}\n
Answer: {answer}\n
Response::: """)

In [38]:
groundness = []
relevance = []
coherence = []
for i in range(len(df)):
    question = df.iloc[i]['questions']
    answer = df.iloc[i]['graph_results']
    context = df.iloc[i]['graph_source_documents']
    groundness_chain = LLMChain(llm=llm, prompt=groundedness_critique_prompt)
    groundness.append(groundness_chain.run(context=context, answer = answer))
    relevance_chain = LLMChain(llm=llm, prompt=relevance_critique_prompt)
    relevance.append(relevance_chain.run(question=question, answer = answer, context = context))
    coherence_chain = LLMChain(llm=llm, prompt=coherence_critique_prompt)
    coherence.append(coherence_chain.run(question=question, answer = answer))

c:\Users\KIIT\anaconda3\envs\myenv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
c:\Users\KIIT\anaconda3\envs\myenv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [39]:
#adding the three lists groundness, relevance and standalone to the dataframe
df['groundness'] = groundness
df['relevance'] = relevance
df['coherence'] = coherence
df.head()

,questions,graph_results,graph_source_documents,groundness,relevance,coherence
0,How did the economic policies of the Ancien Ré...,The economic policies of the Ancien Régime con...,\ntext: % of the population by 1789. Although ...,Evaluation: The answer accurately describes ho...,[Evaluation: The answer provides a detailed ex...,[Evaluation: The answer provides a clear and s...
1,In what ways did the social and political stru...,The social and political structure of the Esta...,"\ntext: During the French Revolution, the Nati...",Evaluation: The answer accurately describes ho...,[Evaluation: The answer provides a detailed ex...,\nEvaluation: The answer provides a clear and ...
2,What role did economic depression and military...,Economic depression and military defeats playe...,\ntext: % of the population by 1789. Although ...,Evaluation: The answer accurately describes ho...,[Evaluation: The answer provides a detailed ex...,\nEvaluation: The answer provides a clear and ...
3,How did the French Revolutionary Wars affect t...,The French Revolutionary Wars had a significan...,\ntext: The French Revolutionary Wars (French:...,Evaluation: The answer accurately describes th...,[Evaluation: The answer provides a detailed ex...,\nEvaluation: The answer provides a clear and ...
4,Examine the socio-economic reasons behind the ...,The calling of the Estates-General in 1789 was...,\ntext: % of the population by 1789. Although ...,[Evaluation: The answer accurately identifies ...,[Evaluation: The answer provides a detailed ex...,[Evaluation: The answer provides a clear and s...


In [40]:
print(df.iloc[0]['relevance'])

[Evaluation: The answer provides a detailed explanation of how the economic policies of the Ancien Régime, such as varying tax rates, inconsistent tax collection, and the complexity of the tax system, contributed to the financial crisis that led to the French Revolution. It also mentions the attempts at reform being blocked by regional Parlements, leading to resentment among taxpayers and the ultimate inability to manage the economic crisis effectively. The answer covers all the key aspects of the question within the context provided.
Total rating: 5]




In [42]:
#printing the content of each column in the first row of the dataframe, separating the name of the column from the content of the cell
print('QUESTION')
print(df.iloc[0]['questions'])
#printing whitespace
print('')
print('ANSWER')
print(df.iloc[0]['graph_results'])
print('')
print('CONTEXT')
print(df.iloc[0]['graph_source_documents'])
print('')
print('GROUNDNESS')
print(df.iloc[0]['groundness'])
print('')
print('RELEVANCE')
print(df.iloc[0]['relevance'])
print('')
print('STANDALONE')
print(df.iloc[0]['coherence'])
     

QUESTION
How did the economic policies of the Ancien Régime contribute to the financial crisis that precipitated the French Revolution?

ANSWER
The economic policies of the Ancien Régime contributed to the financial crisis that precipitated the French Revolution in several ways. The state's debt crisis was exacerbated by varying tax rates across regions, inconsistent tax collection, and the complexity of the tax system. This led to uncertainty over the actual tax contributions and caused resentment among taxpayers. Attempts to reform and simplify the system were blocked by regional Parlements, which approved financial policies. The resulting impasse and the inability to effectively manage the economic crisis, along with other social, political, and cultural factors, ultimately led to the revolution.

CONTEXT

text: % of the population by 1789. Although the 18th century was a period of increasing prosperity, the benefits were distributed unevenly across regions and social groups. Those 

In [44]:
# Save the DataFrame to an Excel file
df.to_excel('graphrag_results.xlsx', index=False)